# 文法开发

我们尝试来进行覆盖更广泛的文法开发。

## 树库和文法

nltk.corpus.treebank 包含了宾州树库语料的 10% 样本：

In [1]:
import nltk
from nltk.corpus import treebank

t = treebank.parsed_sents('wsj_0001.mrg')[0]
print(t)

(S
  (NP-SBJ
    (NP (NNP Pierre) (NNP Vinken))
    (, ,)
    (ADJP (NP (CD 61) (NNS years)) (JJ old))
    (, ,))
  (VP
    (MD will)
    (VP
      (VB join)
      (NP (DT the) (NN board))
      (PP-CLR (IN as) (NP (DT a) (JJ nonexecutive) (NN director)))
      (NP-TMP (NNP Nov.) (CD 29))))
  (. .))


我们可以利用这些数据来帮助开发一个文法。例如在下面的例子中，我们使用一个简单的过滤器找出带句子补语的动词。假设我们已经有一个形如 VP -> V S 的产生式，这个信息使我们能够识别那些包含在 V 的扩张中的特别的动词。

In [2]:
def filter(tree):
    child_nodes = [child.label() for child in tree
                  if isinstance(child, nltk.Tree)]
    return (tree.label() == 'VP') and ('S' in child_nodes)

print([subtree for tree in treebank.parsed_sents()
              for subtree in tree.subtrees(filter)][0])

(VP
  (VBN named)
  (S
    (NP-SBJ (-NONE- *-1))
    (NP-PRD
      (NP (DT a) (JJ nonexecutive) (NN director))
      (PP
        (IN of)
        (NP (DT this) (JJ British) (JJ industrial) (NN conglomerate))))))


nltk.corpus.ppattach（PP 附着语料库）是另一个有关特别动词配价的信息源。这里我们演示挖掘这个语料库的技术，它找出具有固定的介词和名词的介词短语对，其中介词短语附着到 VP 还是 NP，由选择的动词决定：

In [3]:
from collections import defaultdict

entries = nltk.corpus.ppattach.attachments('training')
table = defaultdict(lambda: defaultdict(set))
for entry in entries:
    key = entry.noun1 + '-' + entry.prep + '-' + entry.noun2
    table[key][entry.attachment].add(entry.verb)

for key in sorted(table)[:1000]:
    if len(table[key]) > 1:
        print(key, 'N:', sorted(table[key]['N']), 'V:', sorted(table[key]['V']))

%-below-level N: ['left'] V: ['be']
%-from-year N: ['was'] V: ['declined', 'dropped', 'fell', 'grew', 'increased', 'plunged', 'rose', 'was']
%-in-August N: ['was'] V: ['climbed', 'fell', 'leaping', 'rising', 'rose']
%-in-September N: ['increased'] V: ['climbed', 'declined', 'dropped', 'edged', 'fell', 'grew', 'plunged', 'rose', 'slipped']
%-in-week N: ['declined'] V: ['was']
%-to-% N: ['add', 'added', 'backed', 'be', 'cut', 'go', 'grow', 'increased', 'increasing', 'is', 'offer', 'plummet', 'reduce', 'rejected', 'rise', 'risen', 'shaved', 'wants', 'yield', 'zapping'] V: ['fell', 'rise', 'slipped']
%-to-million N: ['declining'] V: ['advanced', 'climbed', 'cutting', 'declined', 'declining', 'dived', 'dropped', 'edged', 'fell', 'gained', 'grew', 'increased', 'jump', 'jumped', 'plunged', 'rising', 'rose', 'slid', 'slipped', 'soared', 'tumbled']
1-to-21 N: ['dropped'] V: ['dropped']
1-to-33 N: ['gained'] V: ['dropped', 'fell', 'jumped']
1-to-4 N: ['added'] V: ['gained']
1-to-47 N: ['jumped']

NLKT 语料库也手机了 10000 句已分析的中文句子，来自现代汉语中央研究院平衡语料库：

In [4]:
nltk.corpus.sinica_treebank.parsed_sents()[3450].draw()  

![sinica-tree.png](resources/sinica-tree.png)

## 有害的歧义

不幸的是，随着文法覆盖范围的增加和输入句子长度的增长，分析树的数量也迅速增加。例如，我们可以造这样的句子：fish fish fish，意思是 fish like to fish for other fish。我们构造一个关于 fish 的玩具文法，并使用前面提到的图表分析器 [nltk.ChartParser](https://www.nltk.org/_modules/nltk/parse/chart.html#ChartParser) 解析：

In [5]:
grammar = nltk.CFG.fromstring("""
    S -> NP V NP
    NP -> NP Sbar
    Sbar -> NP V
    NP -> 'fish'
    V -> 'fish'
""")

tokens = ['fish'] * 5
cp = nltk.ChartParser(grammar)
for tree in cp.parse(tokens):
    print(tree)

(S (NP fish) (V fish) (NP (NP fish) (Sbar (NP fish) (V fish))))
(S (NP (NP fish) (Sbar (NP fish) (V fish))) (V fish) (NP fish))


随着句子的长度增加到（3，5，7，……），我们得到的分析树的数量是（1，2，5，14，132，429，1430，4862，16796，58786，208012，……）。对于一个长度为 50 的句子有超过 10 的 12 次方种解析方式，没有实际的自然语言处理系统可以为一个句子构建如此多的树，并根据上下文选择一个合适的，但是人类却可以毫不费力地处理这些句子。

前面说到的是**结构歧义**，另一种歧义是**词汇歧义**。只要我们试图建立一个广泛覆盖的文法，我们就被迫使词汇条目对它们的词性高度含糊，例如：dog 既可以是名词，也可以是动词；runs 既可以是动词，也可以是名词；所有的词都可以作为名字被引用；大多数的名词也都可以动词化。因此，一个覆盖广泛的文法分析器将对歧义不堪重负。

歧义在语言中是不可避免的，导致我们的文法在分析看似平淡无奇的句子时低效得可怕。**概率分析**提供了解决这些问题的方法，它使我们能够以来自语料库的证据为基础对歧义句的解析进行可靠性的排序。

## 加权文法

正如我们刚看到的，处理歧义是开发广泛覆盖的分析器的主要挑战。图标分析器提高了计算一个句子的多个分析的效率，但仍然因可能的分析数量过多而不堪重负。**加权文法**和**概率分析算法**为这些问题提供了一个有效的解决方案。

首先，我们需要理解符合语法的概念可能是有倾向性的，可以通过概率分析来处理的。思考动词 give，它既需要一个直接宾语（被给予的东西），也需要一个间接宾语（收件人）。这些补语可以按任何顺序出现:

    a. Kim gave a bone to the dog.

    b. Kim gave the dog a bone.

在“介词格”的形式（a）中，直接宾语先出现，然后是包含间接宾语的介词短语；在“双宾语”的形式（b）中，间接宾语先出现，然后是直接宾语。在这种情况下，两种顺序都是可以接受的。然而，如果简介宾语是代词，人们强烈偏好双宾语结构：

    a. Kim gives the heebie-jeebies to me (介词格).

    b. Kim gives me the heebie-jeebies (双宾语).

使用宾州树库样本，我们可以检查包含 give 的所有介词格和双宾语实例：

In [6]:
def give(t):
    return t.label() == 'VP' and len(t) > 2 and t[1].label() == 'NP'\
            and (t[2].label() == 'PP-DTV' or t[2].label() == 'NP')\
            and ('give' in t[0].leaves() or 'gave' in t[0].leaves())
        
def sent(t):
    return ' '.join(token for token in t.leaves() if token[0] not in '*-0')

def print_node(t, width):
    output = "%s %s: %s / %s: %s" % (sent(t[0]), t[1].label(), sent(t[1]), t[2].label(), sent(t[2]))
    if len(output) > width:
        output = output[:width] + '...'
    print(output)
    
for tree in nltk.corpus.treebank.parsed_sents():
    for t in tree.subtrees(give):
        print_node(t, 72)

gave NP: the chefs / NP: a standing ovation
give NP: advertisers / NP: discounts for maintaining or increasing ad sp...
give NP: it / PP-DTV: to the politicians
gave NP: them / NP: similar help
give NP: them / NP: 
give NP: only French history questions / PP-DTV: to students in a Europe...
give NP: federal judges / NP: a raise
give NP: consumers / NP: the straight scoop on the U.S. waste crisis
gave NP: Mitsui / NP: access to a high-tech medical product
give NP: Mitsubishi / NP: a window on the U.S. glass industry
give NP: much thought / PP-DTV: to the rates she was receiving , nor to ...
give NP: your Foster Savings Institution / NP: the gift of hope and free...
give NP: market operators / NP: the authority to suspend trading in futu...
gave NP: quick approval / PP-DTV: to $ 3.18 billion in supplemental appr...
give NP: the Transportation Department / NP: up to 50 days to review any...
give NP: the president / NP: such power
give NP: me / NP: the heebie-jeebies
give NP: holders / NP: 

我们可以观察到一种强烈的倾向就是最短的补语最先出现，这些偏好可以用加权文法来表示。

**概率上下文无关文法**（probabilistic context free grammar，PCFG）是一种上下文无关文法，它的每一个产生式关联一个概率。它会产生与相应的上下文无关文法相同的文本解析，并给每个解析分配一个概率，概率仅仅是它用到的产生式概率的乘积。

可以使用 [nltk.PCFG](https://www.nltk.org/_modules/nltk/grammar.html#PCFG) 定义加权文法，每个产生式的权值出现在方括号中，**给定左侧的产生式概率之和必须为 1**，多个产生式既可以分开写也可以组合成一行。[nltk.ViterbiParser](https://www.nltk.org/_modules/nltk/parse/viterbi.html#ViterbiParser) 则是用动态规划实现的一个 PCFG 解析器：

In [7]:
grammar = nltk.PCFG.fromstring("""
    S -> NP VP [1.0]
    VP -> TV NP [0.4] | IV [0.3] | DatV NP NP [0.3]
    TV -> 'saw' [1.0]
    IV -> 'ate' [1.0]
    DatV -> 'gave' [1.0]
    NP -> 'telescopes' [0.8]
    NP -> 'Jack' [0.2]
""")

viterbi_parser = nltk.ViterbiParser(grammar)
for tree in viterbi_parser.parse(['Jack', 'saw', 'telescopes']):
    print(tree)

(S (NP Jack) (VP (TV saw) (NP telescopes))) (p=0.064)
